In [ ]:
import itertools
import pprint
from pyspark import SparkConf, SparkContext

#initiating spark context
conf = SparkConf()
sc = SparkContext(conf=conf)


#function to get total number of item Id
def _single(line):
    return [ (single, 1) for single in line.split() ]


#function to create set based on the input setSize
def _set(line, setSize):
    return [ ((itemSet[0], itemSet), 1) for itemSet in list(itertools.permutations(line.split(), setSize))]

# set support
s  = 100
#number of top items to show
topN  = 5
#file name for the dataset
fileName = 'browsing.txt'

file = sc.textFile(fileName, 8)


#creating filter for support threshold
def support_filter(a):
    if a[1]>s:
        return True
    else:
        return False

#creating function to find different combinations
def mapping(a):
    answer=(a[0][0],(a[0][1],a[1]))
    return answer

#creating function for calculating confidence
def confidence(a):
    ratio=float(a[1][0][1]/a[1][1])
    answer=(a[1][0][0],ratio)
    return answer

#creating function used while printing in descending order
def descending(a):
    return (-a[1])

singleCount = (file
                  .flatMap(_single)
                  .reduceByKey(lambda a, b: a + b)
                  .filter(support_filter)
                  .cache()
                  )

confidence = (file
                  .flatMap(lambda line: _set(line, setSize=2))
                  .reduceByKey(lambda a, b: a + b)  #using reduceByKey for better performance
                  .map(mapping)
                  .join(singleCount)
                  .map(confidence) 
                  .cache()
                  )

print ("Top "+str(topN))
pp = pprint.PrettyPrinter(indent=4)
pp.pprint (confidence.takeOrdered(topN, descending))

In [ ]:
sc.stop()